In [4]:
from IPython.html.services.config import ConfigManager
from IPython.paths import locate_profile
cm = ConfigManager(profile_dir=locate_profile(get_ipython().profile))
cm.update('livereveal', {
              'theme': 'solarized',
              'transition': 'slide',
              'start_slideshow_at': 'selected',
              'progress': 'true',
})

{'progress': 'true',
 'start_slideshow_at': 'selected',
 'theme': 'solarized',
 'transition': 'slide'}

<center>
<h2>Let's start</h2>
</center>

<h3>Structured prediction</h3>

<p style="float: left;">Given a sentence $\mathbf{x}=[x_1...x_N]$,<br> predict a graph $\mathbf{G}=(V,E)$<br></p>  <img src="images/tikz/semParse.png" style="width:50%; float: right;">

<p style="float: left;">Given a meaning representation,<br> predict a sentence $\mathbf{x}=[x_1...x_N]$<br>i.e. a linear chain</p>  <img src="images/nlg.png" style="width:50%; float: right;">

Combinations of the two directions cover a wide range of applications: dialog systems, machine translation, etc.

### Two main paradigms

Joint modeling 
- global
- structured

Incremental modeling
- greedy
- pipeline
- transition-based
- history-based

### Joint modeling

A model (e.g. conditional random field) that scores complete outputs (e.g. label sequences):

$$\mathbf{\hat y} =\hat y_{1}\ldots \hat y_{N} = \mathop{\arg \max}_{Y \in {\cal Y}^N} f(y_{1}\ldots y_{N};\mathbf{x})$$

<ul class="fragment">
					<li>no error propagation</li>
					<li>exhaustive exploration of the search space</li>
					<li>large/complex search spaces are challenging</li>
					<li>efficient dynamic programming restricts modelling flexibility
						(i.e. Markov assumptions)</li>
				</ul>


### Incremental modeling

A classifier that predicts one label at a time given the previous predictions:


\begin{align}
\hat y_1 &=\mathop{\arg \max}_{y \in {\cal Y}} f(y;\mathbf{x}),\\
\mathbf{\hat y} = \quad \hat y_2 &=\mathop{\arg \max}_{y \in {\cal Y}} f(y;\mathbf{x}, \hat y_1), \cdots\\
\hat y_N &=\mathop{\arg \max}_{y \in {\cal Y}} f(y;\mathbf{x}, \hat y_{1} \ldots \hat y_{N-1})
\end{align}

<ul class="fragment">
					<li>use our favourite classifier</li>
					<li>no restrictions on features</li>
					<li>prone to error propagation (i.i.d. assumption broken)</li>
					<li>local model not trained wrt the task-level loss</li>
				</ul>


### Imitation learning

Improve incremental modeling to:
- address error-propagation
- train wrt the task-level loss function

**Meta-learning**: use our favourite classifier and features,
but generate better (non-i.i.d.) training data

But let's see some basic concepts first

### Transition system

### Task loss

Needs complete outputs

mention Hamming loss and BLEU as examples

### Supervision for inceremental structured prediction

Usually what we have is labeled data, e.g.:

What can we do with it?

### Expert policy

A function that cheats (oracle) by looking at the labels

Only available for the training data

$\pi^{\star}$

Data as demonstrator, learning from demonstrations

### Action level supervision

- labeled words in PoS tagging (obvious to infer)
- oracle labels for transitions in dependency parsing (not so obvious)

Also known as (expert) demonstrations/trajectories

Show examples to reinforce the fact that we have sequence labeling

it is possible to have dynamic experts

### Per-action reward

Incorporate the effect a mistaken label has on the loss

### Decomposability (or not!)

### Cost-sensitive classification